In [32]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [33]:
f = sc.wholeTextFiles('s3://cpsc5330s21/data-input/textcorpora-SMALL/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
import re
def filename(path):
  return re.match('s3://cpsc5330s21/data-input/textcorpora-SMALL/(.+)\.txt', path)[1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
doc_id_and_all_its_terms = f.map(lambda t: (filename(t[0]), t[1].split()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def tokenPairs(docid, tokens):
  out = []
  for token in tokens:
    item = "".join(filter(lambda c: 97 <= ord(c) <= 122, token.lower()))
    if item != "":
        out.append((docid, item,))
  return out

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
doc_id_term_token = doc_id_and_all_its_terms.flatMap(lambda t: tokenPairs(t[0], t[1])) # [('austen-emma', 'emma'), ('austen-emma', 'by'), ('austen-emma', 'jane'),

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# make term count per document
doc_id_term_token_counts = doc_id_term_token.map(lambda item: ((item[0] ,item[1]), 1,)).reduceByKey(lambda a,b: a+b) # [(('austen-emma', 'emma'), 751), (('austen-emma', 'by'), 567), (('austen-emma', 'austen'), 1),
doc_id_term_token_counts.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(('austen-emma', 'emma'), 751), (('austen-emma', 'by'), 567), (('austen-emma', 'austen'), 1), (('austen-emma', 'volume'), 3), (('austen-emma', 'woodhouse'), 263)]

In [39]:
# count all the terms in the document
docid_term_count = doc_id_term_token.map(lambda item: (item[0], 1,)).reduceByKey(lambda a,b: a+b) # [('carroll-alice', 26382), ('austen-emma', 158128), ('bible-kjv', 790029), 
docid_term_count.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('carroll-alice', 26382), ('austen-emma', 158128), ('bible-kjv', 790029), ('blake-poems', 6817), ('burgess-busterbrown', 15864)]

In [40]:
# form term idf
term_idf = doc_id_term_token.distinct().map(lambda t: (t[1],1)).reduceByKey(lambda a,b: a+b).sortByKey(ascending=True) #  [('a', 10), ('aaron', 2), ('aaronites', 1), ('aarons', 2), ('aback', 1)]
term_idf.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('a', 10), ('aaron', 2), ('aaronites', 1), ('aarons', 2), ('aback', 1)]

In [41]:
# combined function to form tfidf
tfidf = doc_id_term_token_counts.map(lambda t: (t[0][0],(t[0][1], t[1]),)).\
    join(docid_term_count).\
    map(lambda t: (t[1][0][0],(t[1][0][1], t[0], t[1][1]))).\
    join(term_idf).\
    map(lambda t: (t[0],t[1][1],t[1][0][0],t[1][0][1],t[1][0][2],)).\
    map(lambda t: ((t[3],t[0]), 1000000*((t[2]/t[4])/t[1])))
    
tfidf.take(57)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(('carroll-alice', 'down'), 425.3742029498227), (('bible-kjv', 'down'), 158.2220399504322), (('chesterton-brown', 'down'), 190.81127368852154), (('melville-moby_dick', 'down'), 189.3802282844879), (('milton-paradise', 'down'), 124.16208034263154), (('austen-emma', 'down'), 48.48392863165706), (('blake-poems', 'down'), 325.9824295470474), (('burgess-busterbrown', 'down'), 301.1710651650137), (('whitman-leaves', 'down'), 100.69802036846322), (('carroll-alice', 'rabbithole'), 113.71389583807141), (('carroll-alice', 'very'), 606.4741111363809), (('bible-kjv', 'very'), 36.14494601534318), (('chesterton-brown', 'very'), 288.54387728508135), (('melville-moby_dick', 'very'), 167.87167050148514), (('shakespeare-caesar', 'very'), 65.40115433037393), (('austen-emma', 'very'), 834.0641055909699), (('blake-poems', 'very'), 48.89736443205711), (('burgess-busterbrown', 'very'), 602.3421303300274), (('whitman-leaves', 'very'), 35.70202540336423), (('carroll-alice', 'of'), 1925.555302858009), (('bible

In [42]:
# tfidf with highest values 
tfidf.takeOrdered(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(('austen-emma', 'a'), 1943.3623393706364), (('austen-emma', 'abbey'), 145.45178589497118), (('austen-emma', 'abbeymill'), 44.26793483759992), (('austen-emma', 'abbeyoh'), 6.323990691085703), (('austen-emma', 'abbots'), 6.323990691085703)]

In [43]:
# tfidf with lowest values 
tfidf.takeOrdered(5, lambda x: x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(('bible-kjv', 'has'), 0.12657763196034574), (('bible-kjv', 'forgot'), 0.12657763196034574), (('bible-kjv', 'shade'), 0.14064181328927305), (('bible-kjv', 'share'), 0.14064181328927305), (('bible-kjv', 'its'), 0.14064181328927305)]

In [44]:
from pprint import pprint
def relavence(str_to_check):
    split_string = str_to_check.split()
    tokens = list(filter(lambda t: t != "", ["".join(filter(lambda c: 97 <= ord(c) <= 122, x.lower())) for x in split_string]))
    matched_tfidf = tfidf.filter(lambda t: t[0][1] in (tokens)).map(lambda t: (t[0][0], t[1])).reduceByKey(lambda a,b: int((a+b)/(len(split_string)))).sortBy(lambda t: -t[1]).take(5)
    
    return matched_tfidf

pprint(relavence("dead KING garden!"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('bible-kjv', 51),
 ('carroll-alice', 41),
 ('shakespeare-caesar', 31),
 ('whitman-leaves', 30),
 ('chesterton-brown', 26)]

In [45]:
from pprint import pprint
def relavence_like_example(str_to_check):
    split_string = str_to_check.split()
    tokens = list(filter(lambda t: t != "", ["".join(filter(lambda c: 97 <= ord(c) <= 122, x.lower())) for x in split_string]))
    matched_tfidf = tfidf.filter(lambda t: t[0][1] in (tokens)).map(lambda t: (t[0][0], t[1])).reduceByKey(lambda a,b: int((a+b))).sortBy(lambda t: -t[1]).take(5)
    
    return matched_tfidf

pprint(relavence_like_example("dead KING garden!"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('bible-kjv', 371),
 ('carroll-alice', 342),
 ('blake-poems', 173),
 ('chesterton-brown', 160),
 ('milton-paradise', 112)]